In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ml-bank').getOrCreate()
df = spark.read.csv('/home/workspace/jupyterenv/CUB_Models_V1.1/B8/Agri_Loan/input/AgriModelInputNEW.csv', header = True, inferSchema = True)
df.printSchema()

root
 |-- CUSTOMERID: integer (nullable = true)
 |-- ACCOUNTNO: long (nullable = true)
 |-- RATING: integer (nullable = true)
 |-- CONSTITUTION: string (nullable = true)
 |-- RISK: string (nullable = true)
 |-- ACCTTYPE: string (nullable = true)
 |-- PRODUCT: string (nullable = true)
 |-- OPENDATE: string (nullable = true)
 |-- CLOSEDATE: string (nullable = true)
 |-- EXPIRYDATE: string (nullable = true)
 |-- DIFF_DAYS: integer (nullable = true)
 |-- DIFF_MONTHS: integer (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- NPASTATUS: string (nullable = true)
 |-- COLLATERAL_TYPE: string (nullable = true)
 |-- CUSTOMER_SPAN_IN_DAYS: integer (nullable = true)
 |-- CUSTOMER_SPAN_IN_YEARS: integer (nullable = true)
 |-- CUSTOMER_SPAN_CATEGORY: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- AGE_CATEGORY: string (nullable = true)
 |-- LOAN_CLOSED_IN_DAYS: integer (nullable = true)
 |-- LOAN_CLOSED_CATEGORY: string (nullable =

In [11]:
print("Rows= ",df.count())
print("Columns= ",len(df.columns))

Rows=  1326251
Columns=  144


In [12]:
df=df.drop('CUSTOMERID')
df=df.drop('ACCOUNTNO')

df=df.drop('ACCTTYPE')
df=df.drop('STATUS')
df=df.drop('PRODUCT')
df=df.drop('CLOSEDATE')
df=df.drop('OPENDATE')
df=df.drop('EXPIRYDATE')
df=df.drop('HIGHESTSMADATE')

print("Rows= ",df.count())
print("Columns= ",len(df.columns))

Rows=  1326251
Columns=  135


In [13]:
'''check null values'''
from pyspark.sql.functions import isnull, when, count, col
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

+------+------------+----+---------+-----------+--------+---------+---------------+---------------------+----------------------+----------------------+---+------------+-------------------+--------------------+------------+--------------+---------------------+---------------------+----------------+-------------+--------------+------------------------+------------------------+----------------------+------------------------+----------------+-----------------------+-------------------------+---------------------+--------------------------------+------------------------------------+-------------------------------+-----------------------------------------+----------------+------------------+------------------+-------------------+------------+-----------------+--------------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+----------

In [14]:
print(df.count(),len(df.columns))
CONT_COLS_NAMES=[]
STRING_COLS_NAMES=[]
for i in df.dtypes:
    if i[1] == "string":
        STRING_COLS_NAMES.append(i[0])
    else:
        CONT_COLS_NAMES.append(i[0])
print(STRING_COLS_NAMES)
print(CONT_COLS_NAMES)

1326251 135
['CONSTITUTION', 'RISK', 'LOCATION', 'NPASTATUS', 'COLLATERAL_TYPE', 'CUSTOMER_SPAN_CATEGORY', 'AGE_CATEGORY', 'LOAN_CLOSED_CATEGORY', 'SMA_CATEGORY', 'HIGHESTSMASTATUS']
['RATING', 'DIFF_DAYS', 'DIFF_MONTHS', 'CUSTOMER_SPAN_IN_DAYS', 'CUSTOMER_SPAN_IN_YEARS', 'AGE', 'LOAN_CLOSED_IN_DAYS', 'TOTAL_LOAN_ACC', 'TOTAL_ACTIVE_LOAN_ACC', 'TOTAL_CLOSED_LOAN_ACC', 'PRINCIPAL_AMOUNT', 'INTEREST_RATE', 'PROCESSING_FEE', 'TOTAL_ACTIVE_LOAN_AMOUNT', 'TOTAL_CLOSED_LOAN_AMOUNT', 'TOTAL_LOAN_AMOUNT_PAID', 'TOTAL_BALANCE_YET_TO_PAY', 'TOTAL_COLLATERAL', 'TOTAL_ACTIVE_COLLATERAL', 'TOTAL_INACTIVE_COLLATERAL', 'TOTAL_SECURITY_AMOUNT', 'NO_OF_TIMES_ARREAR_INTEREST_PAID', 'TOTAL_DEBITED_ARREAR_INTEREST_AMOUNT', 'NO_OF_PART_PERIOD_INTEREST_PAID', 'TOTAL_DEBITED_PART_PERIOD_INTEREST_AMOUNT', 'NOOFTIMESSMA2_SMA0', 'NOOFTIMESSMA2_SMA1', 'COMBINEDSTATUSVALUE', 'JLLOANSTATUS', 'OTHERACCSMASTATUS', 'COMBINEDOTHERLOANTXNSTATUS', 'SB-M1-BALANCE-mean', 'SB-M10-BALANCE-mean', 'SB-M11-BALANCE-mean', 'SB-M

In [15]:
'''label'''
from pyspark.ml.feature import StringIndexer
stage_1 = StringIndexer(inputCol= 'CONSTITUTION', outputCol= 'CONSTITUTION_index')
stage_2 = StringIndexer(inputCol= 'RISK', outputCol= 'RISK_index')
stage_3 = StringIndexer(inputCol= 'LOCATION', outputCol= 'LOCATION_index')
stage_4 = StringIndexer(inputCol= 'NPASTATUS', outputCol= 'NPASTATUS_index')
stage_5 = StringIndexer(inputCol= 'COLLATERAL_TYPE', outputCol= 'COLLATERAL_TYPE_index')
stage_6 = StringIndexer(inputCol= 'CUSTOMER_SPAN_CATEGORY', outputCol= 'CUSTOMER_SPAN_CATEGORY_index')
stage_7 = StringIndexer(inputCol= 'AGE_CATEGORY', outputCol= 'AGE_CATEGORY_index')
stage_8 = StringIndexer(inputCol= 'LOAN_CLOSED_CATEGORY', outputCol= 'LOAN_CLOSED_CATEGORY_index')
stage_9 = StringIndexer(inputCol= 'SMA_CATEGORY', outputCol= 'SMA_CATEGORY_index')
stage_10 = StringIndexer(inputCol= 'HIGHESTSMASTATUS', outputCol= 'HIGHESTSMASTATUS_index')


In [16]:
from pyspark.ml.feature import OneHotEncoder, VectorAssembler
import os
stage_11 = OneHotEncoder(inputCols=[
                                stage_1.getOutputCol(),stage_2.getOutputCol(), 
                                stage_3.getOutputCol(),stage_4.getOutputCol(),
                                stage_5.getOutputCol(),stage_6.getOutputCol(),
                                stage_7.getOutputCol(),stage_8.getOutputCol(),
                                stage_9.getOutputCol(),stage_10.getOutputCol(),
    
                                ], 
                                outputCols= [
                                'CONSTITUTION_encoded','RISK_encoded',
                                'LOCATION_encoded',
                                'NPASTATUS_encoded','COLLATERAL_TYPE_encoded',
                                'CUSTOMER_SPAN_CATEGORY_encoded','AGE_CATEGORY_encoded',
                                'LOAN_CLOSED_CATEGORY_encoded','SMA_CATEGORY_encoded',
                                'HIGHESTSMASTATUS_encoded'
                                
                                ])

stage_12 = VectorAssembler(inputCols=[
                                'CONSTITUTION_encoded','RISK_encoded',
                                'LOCATION_encoded',
                                'NPASTATUS_encoded','COLLATERAL_TYPE_encoded',
                                'CUSTOMER_SPAN_CATEGORY_encoded','AGE_CATEGORY_encoded',
                                'LOAN_CLOSED_CATEGORY_encoded','SMA_CATEGORY_encoded',
                                'HIGHESTSMASTATUS_encoded','RATING', 'DIFF_DAYS', 'DIFF_MONTHS', 'CUSTOMER_SPAN_IN_DAYS', 'CUSTOMER_SPAN_IN_YEARS', 'AGE', 'LOAN_CLOSED_IN_DAYS', 'TOTAL_LOAN_ACC', 'TOTAL_ACTIVE_LOAN_ACC', 'TOTAL_CLOSED_LOAN_ACC', 'PRINCIPAL_AMOUNT', 'INTEREST_RATE', 'PROCESSING_FEE', 'TOTAL_ACTIVE_LOAN_AMOUNT', 'TOTAL_CLOSED_LOAN_AMOUNT', 'TOTAL_LOAN_AMOUNT_PAID', 'TOTAL_BALANCE_YET_TO_PAY', 'TOTAL_COLLATERAL', 'TOTAL_ACTIVE_COLLATERAL', 'TOTAL_INACTIVE_COLLATERAL', 'TOTAL_SECURITY_AMOUNT', 'NO_OF_TIMES_ARREAR_INTEREST_PAID', 'TOTAL_DEBITED_ARREAR_INTEREST_AMOUNT', 'NO_OF_PART_PERIOD_INTEREST_PAID', 'TOTAL_DEBITED_PART_PERIOD_INTEREST_AMOUNT', 'NOOFTIMESSMA2_SMA0', 'NOOFTIMESSMA2_SMA1', 'COMBINEDSTATUSVALUE', 'JLLOANSTATUS', 'OTHERACCSMASTATUS', 'COMBINEDOTHERLOANTXNSTATUS', 'SB-M1-BALANCE-mean', 'SB-M10-BALANCE-mean', 'SB-M11-BALANCE-mean', 'SB-M12-BALANCE-mean', 'SB-M13-BALANCE-mean', 'SB-M14-BALANCE-mean', 'SB-M15-BALANCE-mean', 'SB-M16-BALANCE-mean', 'SB-M17-BALANCE-mean', 'SB-M18-BALANCE-mean', 'SB-M19-BALANCE-mean', 'SB-M2-BALANCE-mean', 'SB-M20-BALANCE-mean', 'SB-M21-BALANCE-mean', 'SB-M3-BALANCE-mean', 'SB-M4-BALANCE-mean', 'SB-M5-BALANCE-mean', 'SB-M6-BALANCE-mean', 'SB-M7-BALANCE-mean', 'SB-M8-BALANCE-mean', 'SB-M9-BALANCE-mean', 'M1-BALANCE', 'M10-BALANCE', 'M11-BALANCE', 'M12-BALANCE', 'M13-BALANCE', 'M14-BALANCE', 'M15-BALANCE', 'M16-BALANCE', 'M17-BALANCE', 'M18-BALANCE', 'M19-BALANCE', 'M2-BALANCE', 'M20-BALANCE', 'M21-BALANCE', 'M22-BALANCE', 'M23-BALANCE', 'M24-BALANCE', 'M3-BALANCE', 'M4-BALANCE', 'M5-BALANCE', 'M6-BALANCE', 'M7-BALANCE', 'M8-BALANCE', 'M9-BALANCE', 'M1-CREDIT', 'M10-CREDIT', 'M11-CREDIT', 'M12-CREDIT', 'M13-CREDIT', 'M14-CREDIT', 'M15-CREDIT', 'M16-CREDIT', 'M17-CREDIT', 'M18-CREDIT', 'M19-CREDIT', 'M2-CREDIT', 'M20-CREDIT', 'M21-CREDIT', 'M22-CREDIT', 'M23-CREDIT', 'M24-CREDIT', 'M3-CREDIT', 'M4-CREDIT', 'M5-CREDIT', 'M6-CREDIT', 'M7-CREDIT', 'M8-CREDIT', 'M9-CREDIT', 'M1-DEBIT', 'M10-DEBIT', 'M11-DEBIT', 'M12-DEBIT', 'M13-DEBIT', 'M14-DEBIT', 'M15-DEBIT', 'M16-DEBIT', 'M17-DEBIT', 'M18-DEBIT', 'M19-DEBIT', 'M2-DEBIT', 'M20-DEBIT', 'M21-DEBIT', 'M22-DEBIT', 'M23-DEBIT', 'M24-DEBIT', 'M3-DEBIT', 'M4-DEBIT', 'M5-DEBIT', 'M6-DEBIT', 'M7-DEBIT', 'M8-DEBIT', 'M9-DEBIT'
                                ],outputCol='features')

from pyspark.ml import Pipeline
regression_pipeline = Pipeline(stages= [stage_1, stage_2, stage_3, stage_4, stage_5,
                                        stage_6,stage_7,stage_8,stage_9,stage_10,stage_11,stage_12])
ppmodel = regression_pipeline.fit(df)
ppmodel.save(os.path.join('/home/workspace/jupyterenv/CUB_Models_V1.1/B8/Agri_Loan/TrainedModel','Agri_rfencoding'))
data = ppmodel.transform(df)
print(data.select('features').show(1))

+--------------------+
|            features|
+--------------------+
|(150,[0,5,9,10,11...|
+--------------------+
only showing top 1 row

None


In [ ]:
'''Standard Scaling'''
from pyspark.ml.feature import StandardScaler
standardscaler=StandardScaler().setInputCol("features").setOutputCol("Scaled_features")
scalar_fit=standardscaler.fit(data)
scalar_fit.save(os.path.join('/home/workspace/jupyterenv/CUB_Models_V1.1/B8/Agri_Loan/TrainedModel','Agri_rfscalar_fit'))
data=scalar_fit.transform(data)

data.select("features","Scaled_features").show(5)


In [19]:
train, test = data.randomSplit([0.8, 0.2], seed=12345)


In [21]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'Scaled_features', labelCol = 'label')
rfModel = rf.fit(train)
rfModel.save(os.path.join('/home/workspace/jupyterenv/CUB_Models_V1.1/B8/Agri_Loan/TrainedModel','rfcmodel'))

predictions = rfModel.transform(test)
predictions.select('label', 'prediction', 'probability').show(10)

+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|  1.0|       1.0|[0.13692327395069...|
|  0.0|       0.0|[0.96452403588211...|
|  0.0|       0.0|[0.96452403588211...|
|  0.0|       0.0|[0.96452403588211...|
|  0.0|       0.0|[0.96452403588211...|
|  0.0|       0.0|[0.96452403588211...|
|  0.0|       0.0|[0.96452403588211...|
|  1.0|       1.0|[0.34700126008732...|
|  0.0|       0.0|[0.97755469853456...|
|  1.0|       1.0|[0.26844842966334...|
+-----+----------+--------------------+
only showing top 10 rows



In [22]:

import numpy as np
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
find_max = udf(lambda x: float(np.max(np.abs(x))*100), FloatType())
predictions.select('label', 'prediction',find_max("probability").alias("Score %")).show(10)

+-----+----------+--------+
|label|prediction| Score %|
+-----+----------+--------+
|  1.0|       1.0|86.30767|
|  0.0|       0.0| 96.4524|
|  0.0|       0.0| 96.4524|
|  0.0|       0.0| 96.4524|
|  0.0|       0.0| 96.4524|
|  0.0|       0.0| 96.4524|
|  0.0|       0.0| 96.4524|
|  1.0|       1.0|65.29987|
|  0.0|       0.0|97.75547|
|  1.0|       1.0|73.15516|
+-----+----------+--------+
only showing top 10 rows



In [24]:
prdt=predictions.select('label', 'prediction',find_max("probability").alias("Score %"))
prdt.toPandas().to_csv('/home/workspace/jupyterenv/CUB_Models_V1.1/B8/Agri_Loan/output/LoanClassRating.csv')


In [25]:
'''model_testing'''

print(test.count(),len(test.columns))
predictions.select('label', 'prediction').show()

a=predictions.select('label', 'prediction').toPandas()
print(a)
a.to_csv("/home/workspace/jupyterenv/CUB_Models_V1.1/B8/Agri_Loan/output/RANFOR_test.csv")

265113 157
+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
+-----+----------+
only showing top 20 rows

        label  prediction
0         1.0         1.0
1         0.0         0.0
2         0.0         0.0
3         0.0         0.0
4         0.0         0.0
...       ...         ...
265108    0.0         0.0
265109    0.0         0.0
265110    0.0         0.0
265111    0.0         0.0
265112    0.0         0.0

[265113 rows x 2 columns]


In [26]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol='label', 
    predictionCol='prediction', 
    metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
print('Test Accuracy = ', accuracy)

Test Accuracy =  0.9883709965184657


In [27]:
'''bias'''
predictions = rfModel.transform(train)
print(train.count(),len(train.columns))

predictions.select('label', 'prediction').show()

a=predictions.select('label', 'prediction').toPandas()
print(a)
a.to_csv("/home/workspace/jupyterenv/CUB_Models_V1.1/B8/Agri_Loan/output/RANFOR_Train.csv")

1061138 157
+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 20 rows

         label  prediction
0          0.0         0.0
1          0.0         0.0
2          0.0         0.0
3          0.0         0.0
4          0.0         0.0
...        ...         ...
1061133    0.0         0.0
1061134    0.0         0.0
1061135    0.0         0.0
1061136    0.0         0.0
1061137    0.0         0.0

[1061138 rows x 2 columns]


In [28]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol='label', 
    predictionCol='prediction', 
    metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
print('Train Accuracy = ', accuracy)

Train Accuracy =  0.9883021812431559


In [29]:
'''Feature_Importance'''
print(rfModel.featureImportances)
import pandas as pd

def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

ExtractFeatureImp(rfModel.featureImportances, train, "features").head(10)

(150,[7,23,24,25,26,29,30,31,33,43,45,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,83,84,87,92,95,98,100,101,108,117,122,129,133,136,139,141,142,143,145,149],[5.689720284317364e-05,0.010749237290240826,0.010488047465726914,0.0018600935133311937,0.0002950552442574844,5.8720600154645985e-05,0.0003349872892097552,1.551182782490298e-05,7.48846260717104e-05,4.013896564992295e-05,0.0001701291831258764,0.00018885002387050584,0.00043267625584186034,0.23339867933217304,0.002649511525941612,0.004671758973552998,0.25068477277120504,0.007295635238580056,0.037334936247223435,0.017540102919821172,0.02614192133034034,0.017113436392619037,0.019881487920685986,0.005936507710297547,0.02514797743743951,0.03162153078186315,0.03217787429053652,0.03267250864099414,0.006960162855522599,0.006129545973784588,0.007629310026561342,0.011538129089626831,0.026274000534546343,0.013578873424687143,0.016777078934898132,0.028988536947343872,0.040390936644637505,0.009276027304243378,0

,idx,name,score
30,56,COMBINEDOTHERLOANTXNSTATUS,0.250685
27,53,COMBINEDSTATUSVALUE,0.233399
50,76,SB-M8-BALANCE-mean,0.040391
32,58,SB-M10-BALANCE-mean,0.037335
41,67,SB-M19-BALANCE-mean,0.032673
40,66,SB-M18-BALANCE-mean,0.032178
39,65,SB-M17-BALANCE-mean,0.031622
49,75,SB-M7-BALANCE-mean,0.028989
46,72,SB-M4-BALANCE-mean,0.026274
34,60,SB-M12-BALANCE-mean,0.026142


In [34]:
'''confusion matrix'''
import pyspark.sql.functions as F
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
pred_and_labels = predictions.select(['prediction','label']).withColumn('label',F.col('label').cast(FloatType())).orderBy('prediction')
pred_and_labels=pred_and_labels.select(['prediction','label'])
metrics=MulticlassMetrics(pred_and_labels.rdd.map(tuple))
print('confusion matrix')
print(metrics.confusionMatrix().toArray())

confusion matrix
[[734979.   4597.]
 [  7816. 313746.]]
